In [1]:
import numpy as np
import pandas as pd

In [7]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data2 = data[['text','sentiment']]

In [8]:
data3=data2[data.sentiment!='Neutral']

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
8,Deer in the headlights RT @lizzwinstead: Ben C...,Negative
9,RT @NancyOsborne180: Last night's debate prove...,Negative
10,@JGreenDC @realDonaldTrump In all fairness #Bi...,Negative
11,RT @WayneDupreeShow: Just woke up to tweet thi...,Positive
12,Me reading my family's comments about how grea...,Negative


In [11]:
sentiment_label=data3.sentiment.factorize()
sentiment_label[:5]

(array([0, 0, 0, ..., 0, 1, 0], dtype=int64),
 Index(['Positive', 'Negative'], dtype='object'))

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tweet = data3.text.values # select tweets
tokenizer = Tokenizer(num_words=5000) # Create tokenizer with max numbers 5000
tokenizer.fit_on_texts(tweet) # initialize tokenizer on our tweets
vocab_size = len(tokenizer.word_index) + 1 # vocab size
encoded_docs = tokenizer.texts_to_sequences(tweet) # tweets applied to our tokenizer
padded_sequence = pad_sequences(encoded_docs, maxlen=200) # tweets with 200 len, for lstm

In [13]:
print(tweet[0])
print(encoded_docs[0])

RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
[3, 365, 135, 2459, 1, 716, 2, 45, 65, 258, 42, 215, 7, 4368, 177, 1794, 18, 1339, 1430, 761, 13, 10, 11]


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 32

model = Sequential()

model.add(Embedding(5000, embedding_vector_length, input_length=200) )
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 176,651
Trainable params: 176,651
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
history = model.fit(padded_sequence,sentiment_label[0],
                  validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
269/269 [==============================] - 26s 97ms/step - loss: 0.4652 - accuracy: 0.8060 - val_loss: 0.4004 - val_accuracy: 0.8234
Epoch 2/5
269/269 [==============================] - 26s 96ms/step - loss: 0.3087 - accuracy: 0.8692 - val_loss: 0.3251 - val_accuracy: 0.8616
Epoch 3/5
269/269 [==============================] - 26s 96ms/step - loss: 0.2536 - accuracy: 0.8957 - val_loss: 0.3529 - val_accuracy: 0.8588
Epoch 4/5
269/269 [==============================] - 25s 94ms/step - loss: 0.2204 - accuracy: 0.9066 - val_loss: 0.3530 - val_accuracy: 0.8635
Epoch 5/5
269/269 [==============================] - 25s 93ms/step - loss: 0.1966 - accuracy: 0.9203 - val_loss: 0.3741 - val_accuracy: 0.8719


In [16]:
test_word ="This is very happy"
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

'Negative'